# Projeto 1 - Ciência dos Dados

Nome: Nevton de Castro

Nome: Eduardo Ghellere 

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tweepy
import math
import os.path
import pandas as pd
import json
import re
from random import shuffle

In [3]:
with open('auth.pass') as fp:
    data = json.load(fp)
    
    #autentification
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

In [17]:
#Produto escolhido:
produto = 'heineken'

#Quantidade de mensagens capturadas:
n = 500

#Quantidade de mensagens para a base de treinamento:
t = 300

#Filtro de língua
lang = 'pt'

In [11]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't' and msg.full_text.lower()[2] != ' ':
        msgs.append(msg.full_text.lower())
        i += 1                                            #remove todos os retweets, codigo do filtro foi feito em 2018.
        if i > n:
                break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

______
## Criando um arquivo excel

In [12]:
if not os.path.isfile('./{0}.xlsx'.format(produto)):
                                                           #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))
                                                            #fecha o arquivo
    writer.save()


Com o arquivo excel dos tweets, deve-se avaliar um por um manualmente sé caso é relevante ou irrelevante, criando uma coluna denominada "valor" e atribuindo em cada linha um valor de 1 para relevantes e 0 para irrelevantes.

______
## Separando treinamento e teste na base de dados

In [336]:
dados = pd.read_excel('heineken.xlsx')

dados_treinamento = dados.head(300)

dados_teste = dados.tail(200)

dados_teste.reindex()


irrelevantes_trei = dados_treinamento[dados.valor==0] #tweets irrelevantes de treinamento
relevantes_trei = dados_treinamento[dados.valor==1] #tweets relevantes de treinamento

irrelevantes_teste = dados_teste[dados.valor==0] #tweets irrelevantes de teste
relevantes_teste = dados_teste[dados.valor==1]  #tweets relevantes de teste

rel_trei = relevantes_trei['Treinamento'].tolist()
irr_trei = irrelevantes_trei['Treinamento'].tolist()

rel_teste = relevantes_teste['Treinamento'].tolist()
irr_teste = irrelevantes_teste['Treinamento'].tolist()

print()
print('numero total de tweets relevantes na base de treinamento: {0}'.format(len(relevantes_trei)))
print()
print('numero total de tweets irrelevantes na base de treinamento: {0}'.format(len(irrelevantes_trei)))
print()
print('numero total de tweets relevantes na base de teste: {0}'.format(len(relevantes_teste)))
print()
print('numero total de tweets irrelevantes na base de teste: {0}'.format(len(irrelevantes_teste)))
print()


numero total de tweets relevantes na base de treinamento: 90

numero total de tweets irrelevantes na base de treinamento: 210

numero total de tweets relevantes na base de teste: 25

numero total de tweets irrelevantes na base de teste: 175



C:\Users\nevton\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\nevton\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\nevton\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
C:\Users\nevton\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


_____
## Limpando os tweets

A fim de melhorar o classificador algumas palavras e caracteres podem ser removidos dos tweets, e outros devem ser mantidos, como por exemplo o "`#`", já que o simbálo tem um função nos tweets.

In [340]:
def limpa(tw):
    p = []
    for i in tw:
        punctuation = '[!\-.:?;,_+]'       # mantem o `#`
        pattern = re.compile(punctuation)
        text_subbed = re.sub(pattern, ' ', i.lower())
        p.append(text_subbed)

    tweets_limpo = [] 

    for i in p:
        product = 'heineken'                   # remove o nome do produto 
        text_subbed = re.sub(product, ' ', i)
        tweets_limpo.append(text_subbed)

    rfinal = [] 
    for i in tweets_limpo:         
        e = pd.Series(i.split())    #separa as palavras dos tweets
        for i in e:                 
            rfinal.append(i) 
            
    df = pd.DataFrame(rfinal)      #cria um dataframe para analisar a frequencia de cada palavra
    rdata = df[0].value_counts(True)
    
    return [tweets_limpo, df, rdata]


df_rel =    limpa(rel_trei)[1]
df_irr =    limpa(irr_trei)[1]

value_rel = limpa(rel_trei)[2]
value_irr = limpa(irr_trei)[2]

reli_limpos =    limpa(rel_trei)[0]
irri_limpos =    limpa(irr_trei)[0]

rels_limpos =    limpa(rel_teste)[0]
irrs_limpos =    limpa(irr_teste)[0]


___
### Montando um Classificador Naive-Bayes


In [341]:
def Classificador_Naive_Bayes(tweets):      
    lista =[]
    
    d=-1
    for i in tweets: 
        t=1
        e = pd.Series(i.split())
            
        for w in e:
            
            if w in df_rel[0].tolist():
                r = value_rel[w]  
            
            else:
                r = 0
                
            t = r*t
        t = t*(115/500)       
        o=[]
        o.append(t)         
        lista.append(o)        
        
    d = -1
    for i in tweets:
        t =1
        d =d+1
        e = pd.Series(i.split())
        for w in e: 
            
            
            if w in df_irr[0].tolist():
                r = value_irr[w]
            else:
                r = 0
                
                
            t = r*t
        t = t*(385/500)
        lista[d].append(t)
    
    qual=[]
    m=-1
    for i in lista:
        m=m+1
        if (lista[m][0] > lista[m][1]):
            qual.append('relevante')
        if (lista[m][1] > lista[m][0]):
            qual.append('irrelevante')
        if (lista[m][1] == lista[m][0]):
            qual.append('indefinido')
    return qual


____
## Desempenho do Classificador

____
Como o Classificador permite multiplicar por zero em casos de ausência de palavra, muitos dos tweets recebem zero por cento (0%) de chance de serem relevantes ou irrelevantes, por isso foi criado uma terceira classe de indefinidos.
____

### Erros e indefinidos nos tweets relevantes

In [459]:
erros_rel = 0
ind = 0

x = rels_limpos
c = Classificador_Naive_Bayes(x)

for i in c:
    if i == 'irrelevante':
        erros_rel  += 1
    if i == 'indefinido':
        ind += 1

print('numero total de erros nos relevantes: {0}'.format(erros_rel))
r = int(100*erros_rel/(len(x)))
print('porcentagem de erros: {0}%'.format(r))
print()
print('numero total de indefinidos: {0}'.format(ind))
b = int(100*ind/(len(x)))
print('porcentagem de indefinidos: {0}%'.format(b))

print()
print(len(c))

numero total de erros nos relevantes: 1
porcentagem de erros: 4%

numero total de indefinidos: 23
porcentagem de indefinidos: 92%

25


### Erros e indefinidos nos tweets irrelevantes

In [460]:
erros_rel = 0
ind = 0
x = irrs_limpos
c = Classificador_Naive_Bayes(x)
for i in c:
    if i == 'relevante':
        erros_rel  += 1
    if i == 'indefinido':
        ind += 1
        
print('numero total de erros nos irrelevantes: {0}'.format(erros_rel))
r = int(100*erros_rel/(len(x)))
print('porcentagem de erros: {0}%'.format(r))
print()
print('numero total de indefinidos: {0}'.format(ind))
b = int(100*ind/(len(x)))
print('porcentagem de indefinidos: {0}%'.format(b))

print()
print(len(c))

numero total de erros nos irrelevantes: 0
porcentagem de erros: 0%

numero total de indefinidos: 171
porcentagem de indefinidos: 97%

175


_____
# Resolvendo o problema dos indefinidos
____


Para suavizar o classificador de Naive Bayes deve-se utilizar o `Laplace smoothing` que permite que palavras novas não automaticamente desclassifiquem um tweet como relevante ou irrelevante.
____

In [442]:
def Classificador_Naive_Bayes_laplace(tweets):
    
    lista =[]                  #classifica Naive-bayes com Laplace smooting

    d=-1
    for i in tweets:
        t=1

        e = pd.Series(i.split())
        
        for w in e:
            if w in df_rel[0].tolist():
                r = (df_rel[0].value_counts()[w]+1)/(len(df_rel)*2 + len(df_irr))
            else:
                r = 1/(len(df_rel)*2 + len(df_irr))
            t = r*t
        t=t*(115/500)
        o=[]
        o.append(t)         
        lista.append(o)


    d = -1
    for i in tweets:
        t =1
        d =d+1
        e = pd.Series(i.split())
        for w in e:
            if w in df_irr[0].tolist():
                r = (df_irr[0].value_counts()[w]+1)/(len(df_irr)*2 + len(df_irr))
            else:
                r= 1/(len(df_irr)*2 + len(df_rel))
            t = r*t
        t=t*(385/500)
        lista[d].append(t)
    classificados=[]
    m=-1
    for i in lista:
        m=m+1
        if lista[m][0] > lista[m][1]:
            classificados.append('relevante')
        if lista[m][0] < lista[m][1]:
            classificados.append('irrelevante')
            

    return classificados

____
## Desempenho do novo Classificador

_______
Com o problema dos indefinidos resolvido so resta verificar o desempenho real do classificador final
_______

In [457]:
erros_rel = 0
x = rels_limpos
c = Classificador_Naive_Bayes_laplace(x)
for i in c:
    if i != 'relevante':
        erros_rel  += 1       #erros na classificao de relevantes

print('numero total de erros nos relevantes: {0}'.format(erros_rel))
r = int(100*erros_rel/(len(x)))
print()
print('porcentagem de erros: {0}%'.format(r))
print()
print(len(c))


numero total de erros nos relevantes: 17

porcentagem de erros: 68%

25


In [458]:
erros_irr = 0
x = irrs_limpos
c = Classificador_Naive_Bayes_laplace(x)
for i in c:
    if i != 'irrelevante':
        erros_irr  += 1       #erros na classificao de relevantes

print('numero total de erros nos irrelevantes: {0}'.format(erros_irr))
r = int(100*erros_irr/(len(x)))
print()
print('porcentagem de erros: {0}%'.format(r))
print()
print(len(c))


numero total de erros nos irrelevantes: 26

porcentagem de erros: 14%

175


In [456]:
erros_tot = erros_irr + erros_rel
total = int(100*erros_tot/300)
acertos = 100-total

print()
print('porcentagem de erros: {0}%'.format(total))
print()
print('porcentagem de acertos: {0}%'.format(acertos))
print()


porcentagem de erros: 14%

porcentagem de acertos: 86%



___
# Concluindo

____
O classificador de Naive Bayes com o Laplace smooting teve uma porcentagem de acertos de 86%. Em comparação com o classificador sem o Laplace smoothing que teve 97% de indefinidos é possível afirmar que a técnica de suavizar o classificador permitindo que palavras "novas" não automaticamente desclassifiquem um tweet como relevante ou irrelevante.

Assim, é possível dizer que para uma empresa que quer acompanhar os tweets que mencionam suas marcas, o uso desse classificador pode otimizar esse trabalho filtrando tweets irrelevantes, como spams e tweets que apenas mencionam a marca sem oferecer criticas ou elogios. 
____

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**